In [4]:
from azureml.core import Workspace

# Create or retrieve an existing Azure ML workspace
ws = Workspace.get(name='myworkspace',
                      subscription_id='ae1ba1bc-9919-4ae4-8ed6-2a334b24efd7',
                      resource_group='myresourcegroup',
                      location='eastus')
"""
ws = Workspace.create(name='myworkspace',
                      subscription_id='',
                      resource_group='myresourcegroup',
                      location='eastus')
"""
# Write configuration to the workspace config file
ws.write_config(path='.azureml')

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network with one hidden layer
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # Input layer (784 input features)
        self.fc2 = nn.Linear(128, 10)   # Output layer (10 classes)
        self.relu = nn.ReLU()           # Activation function

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Apply ReLU after the first layer
        x = self.fc2(x)             # Output layer
        return x

# Initialize the neural network
model = SimpleNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # For classification tasks
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

In [6]:
from torchvision import datasets, transforms

# Load MNIST dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=32, shuffle=True)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Reset gradients

        # Forward pass
        outputs = model(inputs.view(-1, 784))  # Flatten input
        loss = criterion(outputs, labels)      # Compute loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

100%|██████████| 9.91M/9.91M [00:00<00:00, 190MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 17.8MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 68.9MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 3.35MB/s]


Epoch 1, Loss: 0.8629542910893758
Epoch 2, Loss: 0.3775488056063652
Epoch 3, Loss: 0.3241955622692903
Epoch 4, Loss: 0.29511439379851023
Epoch 5, Loss: 0.2722803581078847


In [7]:
from azureml.core import Model

# Save the trained model
torch.save(model.state_dict(), 'simple_nn.pth')

# Register the model in Azure
model = Model.register(workspace=ws, model_path='simple_nn.pth', model_name='simple_nn')

# Deploying as a web service requires more steps such as creating a scoring script and environment.

Registering model simple_nn
